# 1 Entendimiento de datos/negocio

**Context:** 

FutAlpes F.C., a club in one of Europe’s top five leagues, aims to strengthen its squad in the 2025 transfer market. The club has a budget of €100M and wants to sign 1 player per zone (goalkeeper, defender, midfielder, forward). The transfer market is highly competitive, with historically high player prices.

**Current Problem:**

Player valuations are often based on intuition or experience of agents and scouts. This creates uncertainty and financial risk in investments.

**Proposal:**

Build a machine learning model to predict a player’s market value based on performance statistics.
Include both traditional metrics (goals, assists, passes, duels) and advanced ones (xG, npxG, xA, PSxG).
Assess data quality and representativeness to ensure a solid foundation for the model.

**Expected Value:**
- Reduce uncertainty in transfer negotiations.
- Identify players offering the best performance-to-price ratio.
- Maximize return on investment while reinforcing the squad within budget.

⚠️ Usar <=Python3.11

## 1.1 Carga de datos

In [4]:
import numpy as np
import pandas as pd

In [49]:
db_location = 'data/datos_entrenamiento_laboratorio1(train_data).csv'

In [50]:
football_df=pd.read_csv(db_location, sep=',', encoding = "UTF-8-SIG")

In [51]:
football_df.shape

(57934, 50)

### 1.1.1 Eliminación de columnas duplicadas

Las siguientes columnas estaban duplicadas entonces las eliminamos del dataframe.

In [52]:
dupe_cols = [
    "xAG.1",
    "Pases_intentados.1",
    "Pases_intentados.2",
    "Pases_progresivos.1",
    "Regates_exitosos.1",
    "xAG.2",
    "xAG.3",
    "Pases_completados.1",
    "Pases_completados.2",
]

football_df_slim = football_df.drop(columns=dupe_cols, errors="raise")
football_df_slim.shape

(57934, 41)

In [53]:
football_df_slim.sample(10)

,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Malos_controles,Perdida_balon,Pases_recibidos,Pases_progresivos_recibidos,Faltas_cometidas,Centros,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date
24466,Enzo Barrenechea,ar ARG,"DM,CM",24-001,"Friday May 23, 2025",0,1,0.0,0.0,0.0,...,1,1,67,0,0,0,1,100.0,NaN,NaN
48759,Jordan Lefort,fr FRA,CB,31-044,"Sunday September 22, 2024",0,0,0.0,0.0,0.0,...,0,0,56,0,1,0,2,100.0,€1.50m,30.06.2026
46216,Jordan Veretout,fr FRA,"DM,AM",32-050,"Sunday April 20, 2025",0,1,0.0,0.0,0.0,...,2,0,31,4,1,1,0,0.0,NaN,NaN
44193,Diego Moreira,pt POR,"AM,WB",20-201,"Sunday February 23, 2025",0,0,0.0,0.0,0.1,...,1,0,36,8,0,3,1,33.3,€18.00m,30.06.2029
37467,Tommaso Baldanzi,it ITA,"FW,DM",22-063,"Sunday May 25, 2025",0,0,0.0,0.0,0.0,...,0,0,2,0,1,0,0,NaN,€12.00m,30.06.2028
12094,Jeremy Monga,NaN,LW,NaN,"Sunday May 25, 2025",0,0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,NaN,NaN,NaN
7100,Jose Sa,pt POR,GK,32-008,"Saturday January 25, 2025",0,0,0.0,0.0,0.1,...,0,0,16,0,0,0,0,NaN,€7.00m,30.06.2027
42756,Yassine Kechta,ma MAR,FW,22-329,"Sunday January 19, 2025",0,2,0.1,0.1,0.0,...,0,2,5,0,0,0,0,0.0,€900k,30.06.2028
38429,Remy Cabella,fr FRA,WB,34-177,"Sunday September 1, 2024",0,0,0.0,0.0,0.0,...,0,0,6,0,0,0,0,NaN,NaN,NaN
30070,Nuno Tavares,pt POR,LB,24-325,"Monday December 16, 2024",0,1,0.0,0.0,0.0,...,3,4,44,1,0,5,0,0.0,€25.00m,30.06.2029


### 1.1.2 Analisis de tipos de datos

In [17]:
!pip install ydata-profiling
!pip install ipywidgets

  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]━━━━━━━ 2/3 [ipywidgets]


In [ ]:
import pandas_profiling

profiling = pandas_profiling.ProfileReport(football_df_slim)
profiling.to_file("football_profile.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  6.55it/s]


In [9]:
football_df_slim.dtypes

Jugador                            object
Nacionalidad                       object
Posicion                           object
Edad                               object
Dia_partido                        object
Goles                               int64
Tiros Totales                       int64
xG                                float64
npxG                              float64
xAG                               float64
Acciones_que_crean_tiros            int64
Pases_intentados                    int64
Pases_progresivos                   int64
Regates_exitosos                    int64
Pases_medios_completados            int64
Pases_largos_completados            int64
xA                                float64
Pases_en_ultimo_tercio              int64
Pases_balon_vivo                    int64
Pases_balon_muerto                  int64
Pases_al_hueco                      int64
Pases_centros                       int64
Pases_completados                   int64
Pases_fuera_de_juego              

In [11]:
# Analisis de la variable "Jugador"
num_players = football_df_slim["Jugador"].nunique()
print("Total distinct players:", num_players)

Total distinct players: 2242


In [73]:
# Fuzzy search para encontrar nombres parecidos
!pip install rapidfuzz
from rapidfuzz import fuzz

unique_names = football_df["Jugador"].unique()

possible_dupes = []
for i, name in enumerate(unique_names):
    for other in unique_names[i+1:]:
        score = fuzz.ratio(name, other)
        if score > 88.5:  
            possible_dupes.append((name, other, score))

for name1, name2, score in possible_dupes:
    print(f"{name1}  <--->  {name2}   (similarity: {score}%)")

Neco Williams  <--->  Nico Williams   (similarity: 92.3076923076923%)
Valentin Rosier  <--->  Valentin Rongier   (similarity: 90.32258064516128%)
Sergio Gomez  <--->  Sergi Gomez   (similarity: 95.65217391304348%)
Dani Rodriguez  <--->  Daniel Rodriguez   (similarity: 93.33333333333333%)
Pablo Marin  <--->  Pablo Mari   (similarity: 95.23809523809523%)
Jon Martin  <--->  Jonas Martin   (similarity: 90.9090909090909%)
Ismael Konate  <--->  Ismael Kone   (similarity: 91.66666666666666%)


In [13]:
# Analisis de la variable "Nacionalidad"
print(football_df_slim["Nacionalidad"].value_counts().to_string())

Nacionalidad
fr FRA     9351
es ESP     8951
it ITA     4436
eng ENG    4430
br BRA     2251
ar ARG     1783
pt POR     1631
nl NED     1479
ci CIV     1417
sn SEN     1226
ma MAR     1120
be BEL      968
dk DEN      818
ng NGA      770
se SWE      762
dz ALG      687
ch SUI      673
gh GHA      668
ml MLI      624
rs SRB      618
de GER      597
cm CMR      588
no NOR      576
hr CRO      558
jp JPN      556
co COL      519
pl POL      497
us USA      469
uy URU      452
sct SCO     435
ie IRL      355
ca CAN      334
wls WAL     281
gn GUI      257
ao ANG      252
ge GEO      244
ua UKR      234
al ALB      219
eg EGY      207
cd COD      204
ro ROU      200
sk SVK      193
tr TUR      192
hu HUN      182
ec ECU      182
ga GAB      171
at AUT      166
is ISL      165
si SVN      154
ve VEN      149
gr GRE      147
cz CZE      144
tn TUN      143
xk KVX      137
mx MEX      133
py PAR      132
ru RUS      123
kr KOR      121
cl CHI      115
tg TOG      115
gw GNB      107
cg CGO     

In [14]:
# Analisis de la variable "Posicion"
print(football_df_slim["Posicion"].value_counts().to_string())

Posicion
CB                9537
FW                6858
CM                5763
GK                3987
DM                3729
RB                3265
LB                3244
AM                3236
LW                2703
RW                2638
WB                1903
LM                1654
RM                1629
DM,CM              529
CM,DM              376
LW,RW              221
AM,FW              211
FW,AM              190
LW,LM              175
RW,LW              160
AM,CM              157
RW,RM              147
RB,CB              141
LM,CM              131
AM,LW              128
CB,RB              127
LW,AM              125
FW,LW              118
RM,CM              118
RW,FW              115
LW,FW              115
CM,RM              112
AM,DM              111
CM,AM              109
FW,RW              108
WB,LB              107
CM,LM              105
RM,RW              105
AM,RW              103
RM,LM              102
LM,RM               98
DM,AM               95
CB,LB               92
WB

In [37]:
# Analisis de variable "Edad" (e.g 20-034, 24-122)
mask = football_df_slim["Edad"].astype(str).str.startswith("20")
subset = football_df_slim.loc[mask, "Edad"]

suffix = subset.str.split("-").str[1].astype(int)
print("Min suffix:", suffix.min())
print("Max suffix:", suffix.max())

Min suffix: 0
Max suffix: 365


In [54]:
# Analisis de variable "Goles"
goles = football_df_slim["Goles"]
print("Min goles:", goles.min())
print("Max goles:", goles.max())

goles.value_counts()
football_df_slim = football_df_slim.sort_values(by="Goles", ascending=False)
print(
    football_df_slim.loc[
        football_df_slim["Goles"] > 5, ["Jugador", "Goles"]
    ].to_string()
)

Min goles: -5
Max goles: 995
                     Jugador  Goles
27143          Sergi Roberto    995
42430          Jaydee Canvot    955
33118     Henrikh Mkhitaryan    910
45061            Desire Doue    887
7706           Neco Williams    871
32240              Saul Coco    862
37399        Amir Richardson    861
36984         Pietro Comuzzo    838
4086         Nicolas Jackson    807
26484         Stefan de Vrij    795
41484           Steve Ngoura    792
12403        Victor Meseguer    689
19962             Kike Salas    663
54110              Junya Ito    663
27555              Isak Hien    663
53488        Lilian Raolisoa    633
47821        Abdoulaye Toure    632
18111        Jorge de Frutos    594
36477     Rolando Mandragora    590
16916  Isaac Palazon Camacho    557
52808        Jonathan Gradit    537
4948          Bernardo Silva    528
10901       Antonee Robinson    509
26396         Marten de Roon    507
5784                   Andre    505
1856                Casemiro    485

In [12]:
# Analisis de variable "Tiros totales"
tiros_totales = football_df_slim["Tiros Totales"]
print("Min Tiros Totales:", tiros_totales.min())
print("Max Tiros Totales:", tiros_totales.max())
print("Mean Tiros Totales:", tiros_totales.mean())

Min Tiros Totales: 0
Max Tiros Totales: 12
Mean Tiros Totales: 0.7924016984844824


In [13]:
# Analisis de variable "xG"
xG = football_df_slim["xG"]
print("Min xG:", xG.min())
print("Max xG:", xG.max())
print("Mean xG:", xG.mean())

Min xG: 0.0
Max xG: 2.9
Mean xG: 0.08863534366693135


In [10]:
# Analisis de variable "market_value"
# See how many distinct market values each player has
# Group and collect the unique values as a Python list
market_values_per_player = (
    football_df_slim.groupby("Jugador")["market_value"]
    .apply(lambda x: list(set(x)))
    .reset_index()
)

print(market_values_per_player.to_string())

                             Jugador                      market_value
0              Aaron Ciammaglichella                           [€700k]
1                    Aaron Cresswell                             [nan]
2                      Aaron Malouda                             [nan]
3                       Aaron Martin                          [€6.50m]
4                     Aaron Ramsdale                [€10.00m, €16.00m]
5                   Aaron WanBissaka                         [€24.00m]
6                       Abakar Sylla                         [€10.00m]
7                      Abdallah Sima                          [€9.00m]
8                    Abde Ezzalzouli                         [€12.00m]
9                        Abdel Abqar                          [€7.50m]
10               Abderrahman Rebbach                           [€700k]
11                       Abdon Prats                          [€1.20m]
12                     Abdou Harroui                          [€1.80m]
13    

In [56]:

total_players_multiple_mv = (
    football_df.groupby("Jugador")["market_value"]
    .nunique()
    .gt(1)  # True if >1
    .sum()  # count how many True
)

print(f"# de jugadores con más de un market value: {total_players_multiple_mv}")

# de jugadores con más de un market value: 174


Teniendo en cuenta los valores y los tipos de datos del dataframe, hallamos estas observaciones en el esquema de datos:
- **Jugador** se representa como ```object``` ("Bruno Fernandes") pero debería ser de tipo ```StringDtype``` ya que es texto puro. (Algunos de estos nombres podrían representar el mismo jugador pero con errores)

- **Nacionalidad** se representa como ```object``` ("eng ENG") pero debería ser de tipo ```StringDtype``` ya que es texto puro pero toca transformarlo para que solo sea "ENG".

- **Posicion** se representa como ```object``` ("DM,AM") pero debería ser transformado en algún tipo de encoder ya que cada jugador puede tener mas de una posición en un mismo partido. (Un candidato puede ser *One-Hot Encoding* de posiciones por partido para luego trasformarlo en datos más significativos).

- **Edad** se representa como ```object``` ("33-209") bajo la nomenclatura de "años-días" para cada partido. Debe ser consolidada en una sola edad tomando la edad máxima en el data set y convietiendola a ```float32```.

- **Dia_partido** se representa como ```object``` ("Friday August 16, 2024") pero debería ser de tipo ```Timestamp```.

- **market_value** se representa como ```object``` ("€50.00m") pero debería ser de tipo ```float32```. Esta variable tiene 174 jugadores que tienen más de un **market_value**. Sin embargo, hay ~2000 jugadores unicos lo cual indica que esta variable debería guardar el valor más reciente en el tiempo para cada jugador.

- **contract_date** se representa como ```object``` ("30.06.2028") pero debería ser de tipo ```Timestamp```.


## 1.2 Consideraciones de negocio

(Aca van descripciones de temas relevantes al negocio)

# 2. Preparación de datos

## 2.1 Unicidad

### 2.1.1 Nombre

In [ ]:
# 1. Ver cantidad de jugadores únicos
print("Jugadores distintos:", football_df_slim["Jugador"].nunique())

Jugadores distintos: 2242


Anteriormente vimos que estos jugadores tenian similitud > 88.5% en el nombre

```
Neco Williams  <--->  Nico Williams   (similarity: 92.3076923076923%)
Valentin Rosier  <--->  Valentin Rongier   (similarity: 90.32258064516128%)
Sergio Gomez  <--->  Sergi Gomez   (similarity: 95.65217391304348%)
Dani Rodriguez  <--->  Daniel Rodriguez   (similarity: 93.33333333333333%)
Pablo Marin  <--->  Pablo Mari   (similarity: 95.23809523809523%)
Jon Martin  <--->  Jonas Martin   (similarity: 90.9090909090909%)
Ismael Konate  <--->  Ismael Kone   (similarity: 91.66666666666666%)
```


In [28]:
pairs = [
    ("Neco Williams", "Nico Williams"),
    ("Valentin Rosier", "Valentin Rongier"),
    ("Sergio Gomez", "Sergi Gomez"),
    ("Dani Rodriguez", "Daniel Rodriguez"),
    ("Pablo Marin", "Pablo Mari"),
    ("Jon Martin", "Jonas Martin"),
    ("Ismael Konate", "Ismael Kone"),
]

for p1, p2 in pairs:
    print(f"\n=== {p1}  <->  {p2} ===")
    subset = football_df_slim[
        football_df_slim["Jugador"].str.lower().isin([p1.lower(), p2.lower()])
    ][["Jugador", "Edad", "Nacionalidad"]]

    print(subset.drop_duplicates().sort_values("Jugador").to_string(index=False))


=== Neco Williams  <->  Nico Williams ===
      Jugador   Edad Nacionalidad
Neco Williams 23-126      wls WAL
Neco Williams 23-294      wls WAL
Neco Williams 23-308      wls WAL
Neco Williams 23-316      wls WAL
Neco Williams 23-319      wls WAL
Neco Williams 23-329      wls WAL
Neco Williams 23-336      wls WAL
Neco Williams 23-287      wls WAL
Neco Williams 23-353      wls WAL
Neco Williams 23-364      wls WAL
Neco Williams 24-008      wls WAL
Neco Williams 24-018      wls WAL
Neco Williams 24-028      wls WAL
Neco Williams 24-035      wls WAL
Neco Williams 24-042      wls WAL
Neco Williams 23-357      wls WAL
Neco Williams 23-281      wls WAL
Neco Williams 24-022      wls WAL
Neco Williams 23-268      wls WAL
Neco Williams 23-276      wls WAL
Neco Williams 23-133      wls WAL
Neco Williams 23-140      wls WAL
Neco Williams 23-162      wls WAL
Neco Williams 23-168      wls WAL
Neco Williams 23-176      wls WAL
Neco Williams 23-191      wls WAL
Neco Williams 23-154      wls WAL
Neco 

Después de hacer validaciones comparando edad y nacionalidad con ayuda de un motro de busqueda, se puede verificar que ningún jugador está realmente presente bajo dos nombres diferentes. Sin embargo, hay un jugador "Dani Rodriguez" que solo tiene un registro en el dataset. Se puede considerar eliminar.

#### Eliminar duplicados
Ahora, queremos eliminar los registros de jugadores con valores identicos. Estos son duplicados inecesarios.

In [74]:
# True duplicates: entire row identical
true_dupes = football_df_slim[football_df_slim.duplicated(keep=False)]

print("Numero de true duplicates:", true_dupes.shape[0])

Numero de true duplicates: 20966


In [ ]:
before = football_df_slim.shape[0]
football_df_slim = football_df_slim.drop_duplicates(keep="first")
after = football_df_slim.shape[0]

print(f"Filas antes: {before}\n Filas después: {after}\n Filas eliminadas: {before - after}")

Filas antes: 57934, Filas después: 47451, Filas eliminadas: 10483


Después de eliminar las filas perfectamente duplicadas, ahora identificamos filas en las que el jugador tiene dos o mas registros en la misma fecha.

In [78]:
# Filtrar duplicados por Jugador + Dia_partido
dupes = football_df_slim[
    football_df_slim.duplicated(subset=["Jugador", "Dia_partido"], keep=False)
]

# Ordenar por jugador y fecha
dupes = dupes.sort_values(["Jugador", "Dia_partido"]).reset_index(drop=True)

print(f"Total de duplicados en Jugador y Dia_partido: {len(dupes)}")

Total de duplicados en Jugador y Dia_partido: 5229


In [48]:
# Antes de eliminación
print("Antes de eliminación:", football_df_slim.shape[0])

# Eliminar filas con valores faltantes
football_df_slim = football_df_slim.drop_duplicates()
print("Después de eliminación:", football_df_slim.shape[0])

Antes de eliminación: 57934
Después de eliminación: 47451


Podemos ver que 

In [39]:
true_dupes = football_df[football_df.duplicated(keep=False)]
print("Number of true duplicates:", true_dupes.shape[0])

Number of true duplicates: 20966
